In [37]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime, openpyxl 
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
import nltk
from nltk.corpus import stopwords  
from nltk.stem import PorterStemmer
ps = PorterStemmer()
sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

#df, code = generate_code_from_db(ctx, qgroup=4028)

In [65]:
df=sql_todf("""
SELECT EMAIL_CODE, EMAIL_DESCRIPTION, RecipientDescription FROM EMAIL_MASTER WHERE EMAIL_CODE LIKE '%23%'
""", ctx.connstr())
df['RecipientDescription']=df['RecipientDescription'].apply(lambda s: re.sub('<.*>','',s))
df
df.to_csv(r'c:\temp\emails.csv', index=False)



In [ ]:
lst = [
'1) Is the CyHy finding associated with a Networked Management Interface per BOD 23-02? :'	
,'2) Is this Networked Management Interface going to be removed from Internet? :'	
,'3) Are Zero Trust Principles currently implemented following OMB M-22-09, NIST 800-207, the TIC 3.0 Capability Catalog, and CISAs Zero Trust Maturity Model v2. on this Networked Management Interface?:'	
,'4) Does the implementation include: Multifactor authentication (MFA) :'	
,'5) Does the implementation include: Session Logging :'	
,'6) Does the implementation include: Asset management and identification (client endpoint allow listing) :'	
,'7) Does the implementation include: Isolation of critical workloads and strong access policy :'	
,'8) Does the implementation include: Encryption of data in transit :'	
,'9) Estimated Completion Date for removal from internet and/or ZTA implemented:'	 
] 

In [ ]:
sql="""
SELECT COLUMN_NAME
  FROM INFORMATION_SCHEMA.COLUMNS
 WHERE TABLE_NAME LIKE '%23%' AND (ORDINAL_POSITION > 9 AND ORDINAL_POSITION < 19)
ORDER BY ORDINAL_POSITION ASC
"""
df=sql_todf(sql, connstr)  
df['COLUMN_CAPTION']=lst

In [ ]:

df 


In [ ]:
frm="""
            DECLARE @COLUMN_NAME INT =  (SELECT COLUMN_NAME FROM BOD2302 WHERE PK_BOD2302=@PK_BOD2302) """
frm="""
			IF (SELECT COLUMN_NAME FROM BOD2302 WHERE PK_BOD2302=@PK_BOD2302) = NULL
			BEGIN
				INSERT INTO  @ErrorHolder 		
				SELECT 56500, 'Response is required for COLUMN_CAPTION row ' + @UniqueIdPoamId + ' ' AS Error
			END""" 

s='' 
for i,r in df.iterrows():
    s=s+frm.replace('COLUMN_NAME', r['COLUMN_NAME']).replace('COLUMN_CAPTION', r['COLUMN_CAPTION'])  
print(s)

In [ ]:
sheet=pd.read_excel(r'C:\Users\timko\Downloads\BOD_23-02_Template.xlsx', sheet_name='CyberScope Inventory Tab', header=1, usecols='A:Z').reset_index()

In [ ]:
sheet.columns

In [ ]:
for c in sheet.columns:
    isyesno = False
    if 'Yes/No' in c:
        c=c.split('(Yes/No')[0]
        isyesno=True
    print(re.sub('\n','',c))

    continue

    s=re.sub('[^A-Za-z\s]','',c)
    s=' '.join([s.title() for s in s.split(' ')])
    s=re.sub('[^A-Za-z]','',s)
    s=re.sub('NotMandatory|DoesTheImplementation','',s)
 
    r=f",{s} NVARCHAR(4000) NULL"
    if 'date' in s.lower(): 
        r=f",{s} DATETIME NULL"
    if isyesno: 
        r=f",{s} INT NULL"
    
    print(r)

In [ ]:
ep="""EXECUTE sp_addextendedproperty @name = N'PicklistTypes.UsageType',@level0type = N'SCHEMA',@level0name = 'dbo',@level1type = N'Table', @level1name = 'BOD2302',@level2type = 'Column' 
,@level2name = 'COLUMN_NAME',@value = 'YN'; 
"""

ep="""EXECUTE sp_addextendedproperty @name = N'Control.Type',@level0type = N'SCHEMA',@level0name = 'dbo',@level1type = N'Table', @level1name = 'BOD2302',@level2type = 'Column' 
,@level2name = 'COLUMN_NAME',@value = 'RadTextBox'; 
"""


tmp="""const _COLUMN_NAME = $find($(`div[id$=_COLUMN_NAME]`).attr('id'));"""

In [ ]:
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df['DATA_TYPE']=df['DATA_TYPE'].str.upper()  
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].fillna(0).astype(int)
    df['CHARACTER_MAXIMUM_LENGTH']=df['CHARACTER_MAXIMUM_LENGTH'].apply(lambda s: 4000 if s < 0 else s )
    df[cols] = df[cols].astype(str)
    df=df.loc[:, ['COLUMN_NAME','DATA_TYPE','CHARACTER_MAXIMUM_LENGTH' ]]
    return df

df=sql_todf("""
SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BOD2302'  
""", connstr)   
cols2 = set(df.COLUMN_NAME)
df=normalizedf(df) 
df[-5:]
for i,r in df.iterrows():
    if not 'PK_' in r['COLUMN_NAME']: #  and 'INT' in r['DATA_TYPE'].upper():
        print(tmp.replace('COLUMN_NAME', r['COLUMN_NAME']))

In [ ]:

def get_datafield(df):
    txt=[]   
    for i,r in df.iterrows(): 
        COLUMN_NAME=r['COLUMN_NAME'] 
        DATA_TYPE=r['DATA_TYPE']  
        s=f'<telerik:RadTextBox ID="{COLUMN_NAME}" runat="server"/>'
        if 'CHAR' not in DATA_TYPE:
            s=f'<telerik:RadComboBox ID="{COLUMN_NAME}" runat="server"/>'
        txt.append(f'{s}')
    txt='\n'.join(txt)  
    return txt

def get_sql(df,out=False):
    parms=[]
    for i,r in df.iterrows():
        s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()}({r['CHARACTER_MAXIMUM_LENGTH']}) = NULL"
        if r['DATA_TYPE'] == 'INT':
            s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL" 
        parms.append(f',{s}')

    into=', '.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c])
    vals=', @'.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c]) 
    ins=f'\n\n \t\t \t\t INSERT INTO BOD2302 ({into})'
    ins=ins+f'\n \t\t \t\t VALUES (@{vals})'

    parms='\n\t\t'.join(parms)[:] 
    update=', '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])  
    ext='\t\t \t\t AND ' + ' AND '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])

    sel,exp=[],[]
    for i,r in df.iterrows(): 
        sel.append(f", {r['COLUMN_NAME']} ")
        if r['DATA_TYPE'] == 'INT':     
            sel.append(f", (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['COLUMN_NAME']}_Display]"  ) 
    sel='\t\t \t\t'+'\n \t\t \t\t'.join(sel)  
 
    return {'parms':parms,'ins':ins, 'update':update, 'sel':sel, 'ext':ext , 'exp':exp  }

def get_gridcols(df,out=False):
    ldf=df.to_dict(orient='records') 
    code,ddl,dv,sel=[],[],[],[] 
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        t=f.read() 
        for d in ldf: 
            col=d['COLUMN_NAME']
            s=t.replace("{COLUMN_NAME}",d['COLUMN_NAME']) 
 
            if 'INT' in d['DATA_TYPE']:
                s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{col}" DataValueField="YN" runat="server" />')
                ddl.append(f'BindDDL(e, "{col}", "{col}", "{col}")')
                sel.append(f',(SELECT DisplayValue FROM vwPICKLISTS WHERE PK_Picklist={col}) AS {col}')
            else:
                s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{col}" runat="server" />')
                dv.append(f'DirectCast(e.Item.FindControl($"{col}"), RadTextBox).Text = drVal("{col}").ToString()')
                sel.append(f',{col}')
            code.append(f'{s}')     
    code='\n'.join(code) 
    ddl='\n'.join(ddl)
    dv= '\n'.join(dv)
    sel= '\n'.join(sel)[1:]
    with open(f'{ctx.get_dest()}script.aspx', 'w', encoding='UTF-8') as fw:
        fw.write(f'{code} \n\n {ddl} \n\n {dv} \n\n {sel}')
    if out:  print(txt)
    return {'code':code, 'ddl':ddl,'dv':dv,'sel':sel}
  
 
#p = get_params(df, out=False)
#print ( p['exp'] ) 
print (  get_sql( df )['ext']  )
#d=get_params(df, out=False)
#print (  get_params(df)['sel']  )

In [ ]:
sql = get_gridcols(df)
print( sql['code'])

In [ ]:
d=get_params(df, out=False)
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\BOD2302_CRUD.sql', 'r', encoding='UTF-8') as f:
    sql=f.read()
with open(r'CC:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\BOD2302_CRUD.sql', 'w', encoding='UTF-8') as f:
    sql=re.sub('--parms(?s).*--~parms',f'--parms\n{d["parms"]}\n\t\t--~parms', sql)
    sql=re.sub('--sel(?s).*--~sel',f'--sel\n{d["sel"]}\n\t\t--~sel', sql)
    sql=re.sub('--ins(?s).*--~ins',f'--ins\n{d["ins"]}\n\t\t--~ins', sql)
    sql=re.sub('--update(?s).*--~update',f'--update\n{d["update"]}\n\t\t--~update', sql)
    sql=re.sub('--ext(?s).*--~ext',f'--ext\n{d["ext"]}\n\t\t--~ext', sql)
    f.write(sql)
 

In [ ]:
d=get_params(df, out=False)
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\BOD2302_CRUD.sql', 'r', encoding='UTF-8') as f:
    sql=f.read()
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\BOD2302_CRUD.sql', 'w', encoding='UTF-8') as f:
    sql=re.sub('--parms(?s).*--~parms',f'--parms\n{d["parms"]}\n--~parms\n', sql)
    sql=re.sub('--sel(?s).*--~sel',f'--sel\n{d["sel"]}\n--~sel\n', sql)
    sql=re.sub('--ins(?s).*--~ins',f'--ins\n{d["ins"]}\n--~ins\n', sql)
    sql=re.sub('--update(?s).*--~update',f'--update\n{d["update"]}\n--~update\n', sql)
    f.write(sql)
#print(sql)
    #

In [ ]:
sheet=pd.read_excel(r'C:\Users\timko\Downloads\2302.xlsx', sheet_name='CyberScope Inventory Tab', header=0, usecols='A:G', nrows=3 ).reset_index(drop=True) 
lod=sheet.to_dict(orient='records')  

In [ ]:
import uuid 
rec=lod[0].copy()
rec['Unique ID']=  uuid.uuid4().hex
lod.append(rec)
lod


In [30]:
pd.DataFrame(lod).to_excel(r'C:\Users\timko\Downloads\2302_new.xlsx', index = False)

In [43]:
 
results=pd.DataFrame() 
engine = create_engine(ctx.connstr()) 
connection = engine.raw_connection()
cursor = connection.cursor()
try: 
    cursor.execute("BOD2302_CRUD @PK_OrgSubmission=?, @MODE=?, @UserMode=?, @UserId=?", ['147792', 'SELECT', 'Agency', '1838'])
    results =  list(cursor.fetchall())
    cursor.close()
    connection.commit()
finally:
    connection.close()
results

[(707, 147792, 164, 'LEGISLATIVE', '64be6f2e5de7cb37cd4d44c6', 'DOE Host', '199.137.49.229', '07/24/2023', 'Potentially Risky Service Detected: telnet', '23', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '08/04/2023', '08/04/2023', '08/04/2023', None, None, 'IssueToAgency:08/04/2023', 14812, 'Agency In Progress', 1838, 0, None, None, None, 'Agency')]